In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
import mne
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [33]:
tmin, tmax = -1.,1.
path_root = "subjects/"
event_id = dict(T0=1,T1=2,T2=3)

In [ ]:
# read 109 subjects * 14 tasks (except subject 87)
def read_from_local(n):
    raw_files = []
    for i in range(1,n+1):
        if i == 88:
            continue
        sub_folder = path_root+"S"+str(i).zfill(3)+"/S"+str(i).zfill(3)+"R"
        for j in range(1,15):
            full_path = sub_folder+str(j).zfill(2)+".edf"
            raw_files.append(read_raw_edf(full_path,preload=True,stim_channel="auto"))
    return raw_files
            
def read_from_mne(n):
    raw_files = []
    all_cases = [i for i in range(1,15)]
    for i in range(1,n+1):
        if i == 88:
            continue
        for j in range(1,15):
            raw_files.append(eegbci.load_data(i,all_cases))
    return raw_files
            
raw_files = read_from_local(86)


Extracting EDF parameters from subjects/S001/S001R01.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from subjects/S001/S001R02.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from subjects/S001/S001R03.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S001/S001R04.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting

Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from subjects/S003/S003R03.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S003/S003R04.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S003/S003R05.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S00

Extracting EDF parameters from subjects/S005/S005R04.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S005/S005R05.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S005/S005R06.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S005/S005R07.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setti

Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S007/S007R06.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S007/S007R07.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S007/S007R08.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from subjects/S0

Extracting EDF parameters from subjects/S009/S009R07.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S009/S009R08.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S009/S009R09.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S009/S009R10.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setti

Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S011/S011R09.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S011/S011R10.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S011/S011R11.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S0

Extracting EDF parameters from subjects/S013/S013R10.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S013/S013R11.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S013/S013R12.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S013/S013R13.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setti

Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S015/S015R12.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S015/S015R13.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S015/S015R14.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from subjects/S0

Extracting EDF parameters from subjects/S017/S017R13.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)


In [ ]:
#create list of events and annotations of each files
all_events = []
all_anno = []
for raw in raw_files:
    all_events.append(find_events(raw,shortest_event=0,stim_channel='STI 014'))
    all_anno.append(raw.find_edf_events())

In [ ]:
#find index of abnormal baseline
for i in range(len(all_events)):
    if len(all_events[i]) == 1:
        print(i)


In [ ]:
#ignore its
#del all_events[1218:1220]
#del all_anno[1218:1220]

In [ ]:
#seperate task, baseline and their labels
events = []
base_events = []
labels = []
raws = []
count = 0
for i in range(len(all_events)):
    if len(all_events[i]) > 1:
        events.append(all_events[i])
        raws.append(raw_files[i])
        labels.append(int(count%4)+1)
        count += 1
    else:
        base_events.append(all_events[i])

In [ ]:
#most of events have 29 events, so look for an event that has different event number
for i in range(len(all_events)):
    if len(all_events[i]) != 29 and len(all_events[i] > 1):
        print(i,all_events[i].shape)

In [ ]:
#pick all channels (64 electrodes)
picks = pick_types(raw_files[0].info, meg=False, eeg=True, stim=False, eog=False,exclude='bads')
picks

In [ ]:
#create event id and epochs of all tasks
event_id = {'T0':1,'T1':2,'T2':3}
epochs = [Epochs(raws[i],events[i],event_id,tmin,tmax,proj=True,picks=picks,baseline=None,preload=True) for i in range(len(events))]

In [ ]:
#fetch all the data from epochs
all_data = [epochs[i].get_data() for i in range(len(epochs))]
len(all_data)

In [ ]:
#swap axis 1 and 2
for i in range(len(all_data)):
    all_data[i] = np.transpose(all_data[i],(0,2,1))

In [ ]:
#pad each tensor to have the same length, most of data have (29,321,64) shape
dataset = np.zeros((len(all_data),29,321,64))
for i in range(len(all_data)):
    if all_data[i].shape[0] > 29:
        dataset[i][:29,:all_data[i].shape[1],:all_data[i].shape[1]] = all_data[i][:29,:,:]
    else:
        dataset[i][:all_data[i].shape[0],:all_data[i].shape[1],:all_data[i].shape[2]] = all_data[i]
dataset.shape

In [ ]:
#code of input that has different events's number
for e in epochs:
    if e.get_data().shape[0] != 29:
        print(e)

In [ ]:
#normalize data
re_x = dataset.reshape(dataset.shape[0],-1,len(picks))

In [ ]:
re_x -= re_x.mean(axis=0)

In [ ]:
re_x /= re_x.std(axis=0)

In [ ]:
y = np.asarray(labels)

In [ ]:
re_y = np.zeros((y.shape[0],4))
for i in range(re_y.shape[0]):
    re_y[i,]

In [ ]:
re_y.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(re_x,y,test_size=0.1)

In [ ]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
model = models.Sequential()
model.add(layers.LSTM(32,input_shape=(x_train.shape[1:]),return_sequences=True))
model.add(layers.LSTM(16,return_sequences=True))
model.add(layers.LSTM(8,return_sequences=False))
model.add(layers.Dense(4,activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer="rmsprop",metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5,batch_size=128)

In [ ]:
test_loss,test_acc = model.evaluate(x_test,y_test)

In [ ]:
predict = model.predict(x_test)

In [ ]:
predict

In [ ]:
y_test

In [5]:
'''for experiment'''

'for experiment'

In [7]:
events = find_events(raw_cat, shortest_event=0, stim_channel='STI 014')

Trigger channel has a non-zero initial value of 1 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
31131 events found
Event IDs: [1 2 3]


In [11]:
events.shape

(31131, 3)

In [12]:
picks = pick_types(raw_cat.info, meg=False, eeg=True, stim=False, eog=False,exclude='bads')

In [13]:
picks

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63])

In [14]:
event_id = dict(T0=1,T1=2,T2=3)
epochs_cat = Epochs(raw_cat, events, event_id, tmin, tmax, proj=True, picks=picks,baseline=None, preload=True)

31131 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 31131 events and 321 original time points ...
1203 bad epochs dropped


In [16]:
len(epochs_cat)

29928

In [28]:
epochs_data = epochs_cat.get_data()
epochs_data.shape

(29928, 64, 321)

In [29]:
cv = ShuffleSplit(10, test_size=0.15, random_state=42)
cv_split = cv.split(epochs_data)

In [31]:
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)